# Kotak Neo WebSocket Subscription Demo

This notebook shows how to install the `neo-api-client` package and subscribe to Kotak Neo's WebSocket feed for live market data.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

def _read_env(name: str, *, default: str = "", required: bool = False) -> str:
    value = os.getenv(name, default)
    if isinstance(value, str):
        value = value.strip()
    if required and not value:
        prompt = f'Enter value for {name}: ' if required else f'Enter value for {name} (optional): '
        value = input(prompt).strip()
    if required and not value:
        raise ValueError(f'Missing required environment variable: {name}')
    return value

ENVIRONMENT = _read_env('KOTAK_ENVIRONMENT', default='prod') or 'prod'
UCC = _read_env('KOTAK_UCC', required=True)
CONSUMER_KEY = _read_env('KOTAK_CONSUMER_KEY', required=True)
CONSUMER_SECRET = _read_env('KOTAK_CONSUMER_SECRET', required=True)
NEO_FIN_KEY = _read_env('KOTAK_NEO_FIN_KEY')
MOBILE_NUMBER = _read_env('KOTAK_MOBILE_NUMBER', required=True)


In [ ]:
from neo_api_client import NeoAPI, BaseUrl

base_url = BaseUrl(ucc=UCC).get_base_url()
if not isinstance(base_url, str) or not base_url.startswith('http'):
    raise RuntimeError(f'Failed to resolve base URL for UCC {UCC}: {base_url}')

client = NeoAPI(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    environment=ENVIRONMENT,
    access_token=None,
    neo_fin_key=NEO_FIN_KEY,
    base_url=base_url,
)

totp = input('Enter the 6-digit TOTP: ').strip()
login_response = client.totp_login(mobile_number=MOBILE_NUMBER, ucc=UCC, totp=totp)
print('Login status:', login_response)

if login_response.get('data'):
    mpin = input('Enter the 6-digit MPIN for 2FA (leave blank to skip): ').strip()
    if mpin:
        two_fa_response = client.totp_validate(mpin=mpin)
        print('2FA status:', two_fa_response)


In [ ]:
def on_message(message):
    print('[Res]:', message)

def on_error(message):
    print('[OnError]:', message)

def on_open(message):
    print('[OnOpen]:', message)

def on_close(message):
    print('[OnClose]:', message)

client.on_message = on_message
client.on_error = on_error
client.on_open = on_open
client.on_close = on_close
print('Callbacks registered.')

In [ ]:
instrument_tokens = [
    {'instrument_token': '11536', 'exchange_segment': 'nse_cm'},
    # Example for an index subscription:
    # {'instrument_token': 'Nifty 50', 'exchange_segment': 'nse_cm'},
]

try:
    client.subscribe(instrument_tokens=instrument_tokens, isIndex=False, isDepth=False)
    print('Subscription request sent.')
except Exception as exc:
    print('Subscription failed:', exc)
    raise

In [ ]:
# Run this cell when you want to stop receiving updates for the tokens above.
try:
    client.un_subscribe(instrument_tokens=instrument_tokens, isIndex=False, isDepth=False)
    print('Unsubscribe request sent.')
except Exception as exc:
    print('Unsubscribe failed:', exc)